# Run fitsverify

In [1]:
import os
import sys
import re
import shutil
import subprocess as sp
from configparser import ConfigParser
from random import choice, choices
specprod = 'fuji'
specprod_path = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)

## Create input file

In [2]:
fits_files = os.path.join(os.environ['CSCRATCH'], f'{specprod}_fits.txt')
if not os.path.exists(fits_files):
    # os.chdir(specprod_path)
    with open(fits_files, 'w') as out:
        command = ['find', 'calibnight', 'exposures', 'healpix', 'preproc', 'tiles', 'zcatalog', '-type', 'f', '-name', '*.fits', '-or', '-name', '*.fits.gz']
        proc = sp.Popen(command, stdout=out, stderr=sp.DEVNULL, cwd=specprod_path)
        status = proc.wait()
        if os.path.exists(os.path.join(specprod_path, f'exposures-{specprod}.fits')):
            out.write(f'exposures-{specprod}.fits\n')

## List of Regular Expressions

In [6]:
parser = ConfigParser()
parser.read_string("""
[top]
exposures = exposures-fuji\.fits;exposures-SPECPROD.rst
tiles = tiles-fuji\.fits;tiles-SPECPROD.rst

[calibnight]
biasnight = calibnight/[0-9]{8}/biasnight-[brz][0-9]-[0-9]{8}\.fits\.gz;calibnight/NIGHT/biasnight-CAMERA-NIGHT.rst
fiberflatnight = calibnight/[0-9]{8}/fiberflatnight-[brz][0-9]-[0-9]{8}\.fits;calibnight/NIGHT/fiberflatnight-CAMERA-NIGHT.rst
psfnight = calibnight/[0-9]{8}/psfnight-[brz][0-9]-[0-9]{8}\.fits;calibnight/NIGHT/psfnight-CAMERA-NIGHT.rst

[exposures]
cframe = exposures/[0-9]{8}/[0-9]{8}/cframe-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/cframe-CAMERA-EXPID.rst
exposure-qa = exposures/[0-9]{8}/[0-9]{8}/exposure-qa-[0-9]{8}\.fits;exposures/NIGHT/EXPID/exposure-qa-EXPID.rst
fiberflat = exposures/[0-9]{8}/[0-9]{8}/fiberflat-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fiberflat-CAMERA-EXPID.rst
fiberflatexp = exposures/[0-9]{8}/[0-9]{8}/fiberflatexp-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fiberflatexp-CAMERA-EXPID.rst
fit-psf = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-CAMERA-EXPID.rst
fit-psf-before-listed = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-listed-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-before-blacklisted-CAMERA-EXPID.rst
fit-psf-before-listed-fix = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-listed-fix-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-before-blacklisted-fix-CAMERA-EXPID.rst
fit-psf-fixed-listed = exposures/[0-9]{8}/[0-9]{8}/fit-psf-fixed-listed-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-fixed-blacklisted-CAMERA-EXPID.rst
fluxcalib = exposures/[0-9]{8}/[0-9]{8}/fluxcalib-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fluxcalib-CAMERA-EXPID.rst
frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/frame-CAMERA-EXPID.rst
psf = exposures/[0-9]{8}/[0-9]{8}/psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/psf-CAMERA-EXPID.rst
sframe = exposures/[0-9]{8}/[0-9]{8}/sframe-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/sframe-CAMERA-EXPID.rst
shifted-input-psf = exposures/[0-9]{8}/[0-9]{8}/shifted-input-psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/shifted-input-psf-CAMERA-EXPID.rst
sky = exposures/[0-9]{8}/[0-9]{8}/sky-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/sky-CAMERA-EXPID.rst
stdstars = exposures/[0-9]{8}/[0-9]{8}/stdstars-[0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/stdstars-SPECTROGRAPH-EXPID.rst

[healpix]
coadd = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/coadd-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/coadd-SURVEY-PROGRAM-PIXNUM.rst
qso_mgii = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_mgii-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/qso_mgii-SURVEY-PROGRAM-PIXNUM.rst
qso_qn = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_qn-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/qso_qn-SURVEY-PROGRAM-PIXNUM.rst
redrock = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/redrock-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/redrock-SURVEY-PROGRAM-PIXNUM.rst
spectra = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/spectra-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/spectra-SURVEY-PROGRAM-PIXNUM.rst
tilepix = healpix/tilepix\.fits;healpix/tilepix.rst

[preproc]
fibermap = preproc/[0-9]{8}/[0-9]{8}/fibermap-[0-9]{8}\.fits;preproc/NIGHT/EXPID/fibermap-EXPID.rst
preproc = preproc/[0-9]{8}/[0-9]{8}/preproc-[brz][0-9]-[0-9]{8}\.fits;preproc/NIGHT/EXPID/preproc-CAMERA-EXPID.rst

[tiles]
coadd = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/coadd-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/coadd-SPECTROGRAPH-NIGHT-EXPID.rst
qso_mgii = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_mgii-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/qso_mgii-SPECTROGRAPH-NIGHT-EXPID.rst
qso_qn = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_qn-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/qso_qn-SPECTROGRAPH-NIGHT-EXPID.rst
redrock = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/redrock-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/redrock-SPECTROGRAPH-NIGHT-EXPID.rst
spectra = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/spectra-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/spectra-SPECTROGRAPH-NIGHT-EXPID.rst
tile-qa = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/tile-qa-NIGHT-EXPID.rst

[tiles:depth]
coadd = tiles/[14]x_depth/[0-9]+/[0-9]/coadd-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/coadd-SPECTROGRAPH-TILEID-EXPID.rst
qso_mgii = tiles/[14]x_depth/[0-9]+/[0-9]/qso_mgii-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_mgii-SPECTROGRAPH-TILEID-EXPID.rst
qso_qn = tiles/[14]x_depth/[0-9]+/[0-9]/qso_qn-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_sn-SPECTROGRAPH-TILEID-EXPID.rst
redrock = tiles/[14]x_depth/[0-9]+/[0-9]/redrock-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/redrock-SPECTROGRAPH-TILEID-EXPID.rst
spectra = tiles/[14]x_depth/[0-9]+/[0-9]/spectra-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/spectra-SPECTROGRAPH-TILEID-EXPID.rst

[zcatalog]
zpix = zcatalog/zpix-(sv1|sv2|sv3|main)-(backup|bright|dark|other)\.fits;zcatalog/zpix-SURVEY-PROGRAM.rst
ztile = zcatalog/ztile-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-(cumulative|pernight)\.fits;zcatalog/ztile-SURVEY-PROGRAM-TILETYPE.rst

[exclude]
calibnight = calibnight/[0-9]{8}/tmp/*;dummy
exposures = exposures/[0-9]{8}/old/*;dummy
preproc = preproc/[0-9]{8}/old/*;dummy
frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}-no-badcolumn-mask\.fits;dummy
fit = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}_[0-9][0-9]\.fits;dummy
redrock = tiles/pernight/[0-9]+/[0-9]{8}/redrock-tmp-[0-9]-[0-9]+-[0-9]{8}\.fits;dummy
rrdetails = tiles/pernight/[0-9]+/[0-9]{8}/rrdetails-tmp-[0-9]-[0-9]+-[0-9]{8}\.fits;dummy
"""
    )


## Precompile Regular Expressions

In [7]:
r = dict()
structure = dict()
for s in parser.sections():
    r[s] = dict()
    structure[s] = dict()
    for key, value in parser.items(s):
        v = value.split(';')
        r[s][key] = re.compile(v[0])
        structure[s][key] = v[1]

## Scan the list of files

In [8]:
with open(fits_files) as e:
    data = e.readlines()
data.append(f'exposures-{specprod}.fits\n')
data.append(f'tiles-{specprod}.fits\n')
scanable = dict()
for file in data:
    ff = file.strip()
    f = ff.split('/')
    if len(f) == 1:
        section = 'top'
    elif f[1] == '1x_depth' or f[1] == '4x_depth':
        section = 'tiles:depth'
    else:
        section = f[0]
    if section not in scanable:
        scanable[section] = dict()
    excluded = False
    for key in r['exclude']:
        m = r['exclude'][key].match(ff)
        if m is not None:
            excluded = True
    if excluded:
        continue
    matched = False
    for key in r[section]:
        m = r[section][key].match(ff)
        if m is not None:
            matched = True
            if key in scanable[section]:
                scanable[section][key].append(ff)
            else:
                scanable[section][key] = [ff]
    if matched:
        continue
    print("ERROR: Could not match {0}!".format(ff))

ERROR: Could not match tiles/pernight/21127/20210605/priors-tmp-1-21127-20210605.fits!
ERROR: Could not match tiles/pernight/20505/20210627/priors-tmp-9-20505-20210627.fits!
ERROR: Could not match tiles/pernight/20505/20210627/priors-tmp-1-20505-20210627.fits!
ERROR: Could not match tiles/pernight/20418/20210522/priors-tmp-8-20418-20210522.fits!


## From the list of all file types, pick one at random

In [9]:
scan = dict()
for section in scanable:
    scan[section] = dict()
    for ftype in scanable[section]:
        scan[section][ftype] = choice(scanable[section][ftype])
scan

{'calibnight': {'biasnight': 'calibnight/20201216/biasnight-z2-20201216.fits.gz',
  'fiberflatnight': 'calibnight/20210508/fiberflatnight-b0-20210508.fits',
  'psfnight': 'calibnight/20210320/psfnight-b5-20210320.fits'},
 'exposures': {'fit-psf': 'exposures/20210627/00096314/fit-psf-z0-00096314.fits',
  'shifted-input-psf': 'exposures/20210511/00087993/shifted-input-psf-z6-00087993.fits',
  'fit-psf-fixed-listed': 'exposures/20210410/00084173/fit-psf-fixed-listed-b7-00084173.fits',
  'fit-psf-before-listed-fix': 'exposures/20210508/00087654/fit-psf-before-listed-fix-z2-00087654.fits',
  'frame': 'exposures/20210507/00087599/frame-z7-00087599.fits',
  'psf': 'exposures/20210208/00075119/psf-r1-00075119.fits',
  'sframe': 'exposures/20210208/00075104/sframe-r1-00075104.fits',
  'sky': 'exposures/20210514/00088385/sky-b3-00088385.fits',
  'fiberflatexp': 'exposures/20201216/00068273/fiberflatexp-b5-00068273.fits',
  'cframe': 'exposures/20210217/00076656/cframe-r9-00076656.fits',
  'fluxc

## Run fitsverify on the chosen files

In [10]:
for section in scan:
    for key in scan[section]:
        command = ['fitsverify', '-l', scan[section][key]]
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
        out, err = proc.communicate()
        # print(section, key, out.decode('ascii').split('\n')[-2])
        result = out.decode('ascii').split('\n')[-2]
        if result == "**** Verification found 0 warning(s) and 0 error(s). ****":
            pass
            # print(section, key, "No problems detected.")
        else:
            print(section, key, "Problems detected!")
            print(out.decode('ascii'))
            if err:
                print(err.decode('ascii'))

calibnight fiberflatnight Problems detected!
 
              fitsverify 4.18 (CFITSIO V3.410)              
              --------------------------------              
 
 
File: calibnight/20210508/fiberflatnight-b0-20210508.fits

6 Header-Data Units in this file.
 
=================== HDU 1: Primary Array ===================
 
*** Warning: Keyword #202, FRAMES has a null value.
*** Warning: Keyword #213, OPENSHUT has a null value.
 
   1 | SIMPLE  =                    T / conforms to FITS standard
   2 | BITPIX  =                  -32 / array data type
   3 | NAXIS   =                    2 / number of array dimensions
   4 | NAXIS1  =                 2751
   5 | NAXIS2  =                  500
   6 | EXTEND  =                    T
   7 | EXTNAME = 'FIBERFLAT'
   8 | EXPID   =                87667 / Exposure number
   9 | EXPFRAME=                    0 / Frame number
  10 | FLAVOR  = 'science '           / Observation type
  11 | SEQUENCE= 'Spectrographs'      / OCS Sequence name
  12 

## Generate datamodel stub files on all chosen files

In [ ]:
for section in scan:
    for key in scan[section]:
        command = ['generate_model', os.path.join(specprod_path, scan[section][key])]
        print(' '.join(command))
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE)
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(out.decode('ascii'))
            print(err.decode('ascii'))
        if key == 'shifted-input-psf':
            src = 'shifted'
        elif key == 'fit-psf' or key == 'fit-psf-fixed-blacklisted' or key == 'fit-psf-before-blacklisted-fix':
            src = 'fit'
        elif key == 'exposure-qa':
            src = 'exposure'
        elif key == 'tile-qa':
            src = 'tile'
        else:
            src = key
        if os.path.exists(f'{src}.rst'):
            d = os.path.dirname(structure[section][key])
            if d and not os.path.isdir(d):
                print("os.makedirs('{0}')".format(d))
                os.makedirs(d)
            print("shutil.move('{0}.rst', '{1}'".format(src, structure[section][key]))
            shutil.move('{0}.rst'.format(src), structure[section][key])
        else:
            print("ERROR: Could not find file corresponding to {0}.rst -> {1}!".format(src, scan[section][key]))
            

## Scan preproc files for errors

In [22]:
for p in choices(scanable['preproc']['preproc'], k=len(scanable['preproc']['preproc'])//100):
    command = ['fitsverify', '-l', p]
    proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
    out, err = proc.communicate()
    # print(section, key, out.decode('ascii').split('\n')[-2])
    result = out.decode('ascii').split('\n')[-2]
    if result == "**** Verification found 0 warning(s) and 0 error(s). ****":
        print(p, "No problems detected.")
    else:
        # print(p, "Problems detected!")
        o = out.decode('ascii')
        if ('*** Warning: Data checksum is not consistent with  the DATASUM keyword' in o and
            '*** Warning: HDU checksum is not in agreement with CHECKSUM.' in o):
            lines = o.split('\n')
            hduline = lines[lines.index('*** Warning: Data checksum is not consistent with  the DATASUM keyword') - 2]
            print(p, 'Checksum problems detected in {0}!'.format(hduline.strip('=').strip()))
            # print(out.decode('ascii'))
        else:
            print(p, "Other problems detected!")
        # if err:
        #     print(err.decode('ascii'))

preproc/20210410/00084204/preproc-b1-00084204.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210506/00087418/preproc-r8-00087418.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210508/00087669/preproc-r2-00087669.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210408/00083928/preproc-z4-00083928.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210410/00084157/preproc-z7-00084157.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210408/00083949/preproc-b8-00083949.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210314/00080498/preproc-b7-00080498.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210508/00087705/preproc-z4-00087705.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210408/00083929/preproc-z5-00083929.fits Checksum problems detected in HDU 3: BINARY Table!
preproc/20210509/00087761/preproc-r7-00087761.fits Checksum problems detected in H